In [13]:
import json
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, MapType, ArrayType, BooleanType
#Convert data Unix timestamp to datetime
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import count
from pyspark.sql.functions import desc
import pyspark.sql.functions as f


game_api_raw = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","events") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 
game_api_raw.cache()
game_api_raw.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 65 76 65 6...|events|        0|     0|2020-07-12 16:53:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     1|2020-07-12 16:54:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     2|2020-07-12 16:55:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     3|2020-07-12 16:55:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     4|2020-07-12 16:56:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     5|2020-07-12 16:56:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     6|2020-07-12 16:56:...|            0|
|null|[7B 22 65 76 65 6...|events|        0|     7|2020-07-12 16:56:...|            0|
|null|[7B 22 65 76 65 6...|events|        0

In [14]:
game_api_raw.printSchema()
game_api_raw.count()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



136

In [16]:
#Get the 'value' field and cast it as a string 
game_api_string = game_api_raw.select(game_api_raw.value.cast('string'))
game_api_string.show()

+--------------------+
|               value|
+--------------------+
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
|{"event_type": "d...|
+--------------------+
only showing top 20 rows



In [17]:
game_api_df = game_api_string.rdd.map(lambda x: json.loads(x.value)).toDF()
game_api_df.show(100)

+----------+
|event_type|
+----------+
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|
|   default|

/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [27]:
game_api_df.withColumn('timestamp', game_api_raw.column('timestamp')).show()

AttributeError: 'DataFrame' object has no attribute 'column'

In [24]:
game_api_raw.select('timestamp').show()

+--------------------+
|           timestamp|
+--------------------+
|2020-07-12 16:53:...|
|2020-07-12 16:54:...|
|2020-07-12 16:55:...|
|2020-07-12 16:55:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:56:...|
|2020-07-12 16:57:...|
|2020-07-12 16:57:...|
|2020-07-12 16:58:...|
|2020-07-12 16:58:...|
|2020-07-12 16:58:...|
|2020-07-12 16:58:...|
|2020-07-12 16:59:...|
+--------------------+
only showing top 20 rows



In [34]:
from datetime import datetime
str(datetime.now())

'2020-07-16 04:42:30.745807'